In [31]:
#cargando paquetes:
using Pkg
using CSV
using Distributions
using DataFrames
using Dates
using Plots
using Random
using LinearAlgebra
using LaTeXStrings
using Lasso
using Statistics
using GLMNet
using MLJ
using StatsBase
using CSV
using StatsModels
using GLM

In [32]:
#cargando la data:
data = CSV.read("../../../data/wage2015_subsample_inference.csv", DataFrame,
                types = Dict(:occ2 => String, :ind2 => String)) ;

In [33]:
#tenemos nuestro Y:
y = data[:, "lwage"]

Z = select(data, Not([:wage, :lwage, :rownames]))


Row,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String,Float64,String
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,7.0,0.49,0.343,0.2401,3600.0,11,8370.0,18
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,3050.0,10,5070.0,9
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,3.24,5.832,10.4976,6260.0,19,770.0,4
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,25.0,6.25,15.625,39.0625,420.0,1,6990.0,12
5,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,2015.0,6,9470.0,22
6,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.01,0.001,0.0001,1650.0,5,7460.0,14
7,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,42.0,17.64,74.088,311.17,5120.0,17,7280.0,14
8,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,37.0,13.69,50.653,187.416,5240.0,17,5680.0,9
9,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,4040.0,13,8590.0,19


In [34]:
train_sample = rand(Float64, size(data)[1]) .< 0.75
test_sample = .!(train_sample)
y_train, y_test = y[train_sample], y[test_sample]
X_train, X_test = Z_base[train_sample, :], Z_base[test_sample, :];

## modelo flexible

In [35]:
extra_flexible_formula = @formula(lwage ~ 1 + sex + (exp1 + exp2 + exp3 + exp4) * (hsg + scl + clg + ad + so + we + ne + occ2 + ind2) + (hsg + scl + clg + ad) * (so + we + ne + occ2 + ind2) + (so + we + ne) * (occ2 + ind2) + occ2 * ind2)
X_extra_flexible = modelmatrix(extra_flexible_formula, data)

5150×965 Matrix{Float64}:
 1.0  1.0   7.0   0.49   0.343  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  31.0   9.61  29.791     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  18.0   3.24   5.832     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  25.0   6.25  15.625     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  22.0   4.84  10.648     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0   1.0   0.01   0.001  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  42.0  17.64  74.088     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  37.0  13.69  50.653     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  31.0   9.61  29.791     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0   4.0   0.16   0.064     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                              ⋱            ⋮                   
 1.0  0.0  15.0   2.25   3.375     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  11.0   1.21   1.331     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  17.0   2.89   4.913     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  

In [36]:
#8. este es el resultado con una OLS
X_train, X_test = X_extra_flexible[train_sample, :], X_extra_flexible[test_sample, :];

In [37]:
lr_flex = lm(X_train, y_train);

In [38]:
basic_mse_testing = mean((GLM.predict(lr_flex, X_test) - y_test) .^ 2)
basic_r2_testing = 1 - basic_mse_testing / var(y_test)

0.1391996007733225

In [39]:
basic_mse_testing

0.27010485563562864

In [40]:
#para quitar los ceros:
println(size(X_extra_flexible))

(5150, 965)


In [41]:
using DataFrames
using Statistics
using StatsBase
using CSV
using StatsModels
using GLM

In [47]:
train_sample = rand(Float64, size(data)[1]) .< 0.9
test_sample = .!(train_sample)
y_train, y_test = y[train_sample], y[test_sample]
X_extra_flexible_train, X_extra_flexible_test = X_extra_flexible[train_sample, :], X_extra_flexible[test_sample, :];

In [48]:
extra_flexible_model = lm(X_extra_flexible_train, y_train)

LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}:

Coefficients:
──────────────────────────────────────────────────────────────────────────────
             Coef.   Std. Error       t  Pr(>|t|)     Lower 95%      Upper 95%
──────────────────────────────────────────────────────────────────────────────
x1     3.85631        1.29473      2.98    0.0029    1.31789       6.39473
x2    -0.0602473      0.016821    -3.58    0.0003   -0.0932262    -0.0272685
x3    -0.23662        0.282354    -0.84    0.4021   -0.790198      0.316958
x4     2.40491        2.14348      1.12    0.2619   -1.79755       6.60738
x5    -0.867125       0.669247    -1.30    0.1952   -2.17924       0.444986
x6     0.103074       0.0729937    1.41    0.1580   -0.0400365     0.246183
x7    -1.11747        1.24718     -0.90    0.3703   -3.56267       1.32772
x8    -0.874127       1.24147     -0.70    0.4814   -3.30812       1.55987
x9    -0.489764

In [51]:
#normalizamos la variable Y
mean_y = mean(y)
y_centered = y .- mean_y
std_y = std(y)
y_normalized = y_centered ./ std_y


5150-element Vector{Float64}:
 -1.2402545501637507
  1.581415943901389
 -0.9952235701262617
 -0.5888284120346895
  0.6858352078834439
 -0.8916287370734166
 -0.02502721376252582
 -0.02502721376252582
 -0.8518460993196265
 -0.02502721376252582
  ⋮
  0.25770849799681306
 -0.2591347787012701
  0.3661886075001642
 -0.6009613493971904
 -0.48781225590462607
  0.29461938662075393
  1.1902001226386991
  0.9199427728221882
 -0.20974552813450043

## Creating the Lasso Cross-Validation Procedure

In [80]:
#creamos para el grid
 function logarithmically_spaced_grid(limite_inf::Float64, limite_sup::Float64, log_espacio::Float64)
    len_cell = Int(ceil((log(limite_sup) - log(limite_inf)) / log_espacio) + 1)
        
    grid = exp.(range(log(limite_inferior), log(limite_superior), length=len))
    
    return grid
end


logarithmically_spaced_grid (generic function with 1 method)

In [62]:
#haciendo la función para folds
function generar_k_folds(n_rows::Int, k::Int)
    indices = shuffle(1:n_rows)
    
    fold_size = div(n_rows, k)
    remainder = n_rows % k
    
    folds = Vector{Vector{Int}}(undef, k)
    
    start = 1
    for i in 1:k
        extra = i <= remainder ? 1 : 0 
        stop = start + fold_size + extra - 1 
        folds[i] = indices[start:stop]
        start = stop + 1
    end
    
    return folds
end


generar_k_folds (generic function with 1 method)

In [79]:
#7.
using GLMNet

function lasso_cv(X::AbstractArray, y::AbstractArray, limites_grid::Tuple{Float64, Float64, Float64}, n_rows::Int, k::Int)
    # Desempaquetar los límites y el espaciado logarítmico
    logarithmically_spaced_grid()
    
    # para los folds
    indices = shuffle(1:n_rows)
    fold_size = div(n_rows, k)
    remainder = n_rows % k
    folds = Vector{Vector{Int}}(undef, k)
    
    start = 1
    for i in 1:k
        extra = i <= remainder ? 1 : 0
        stop = start + fold_size + extra - 1
        folds[i] = indices[start:stop]
        start = stop + 1
    end
    
    n_lambdas = length(grid)
    mse_per_lambda = zeros(n_lambdas)
    
    for (j, lambda) in enumerate(grid)
        mse_fold = 0.0
        
        for fold in folds
            X_train = X[.!fold, :]
            y_train = y[.!fold]
            X_test = X[fold, :]
            y_test = y[fold]
            
            
            model = glmnet(X_train, y_train, alpha=1.0, lambda=[lambda])# lasso
            
           
            y_pred = GLMNet.predict(model, X_test) # usando predic y GLMNet
            
            
            mse_fold += mean((y_pred .- y_test).^2) # MSE
        end
        
        # Almacenar el MSE promedio para el lambda actual
        mse_per_lambda[j] = mse_fold / k
    end
    
    # Encontrar el lambda óptimo con el MSE mínimo
    optimal_lambda_idx = argmin(mse_per_lambda)
    optimal_lambda = grid[optimal_lambda_idx]
    
    # Ajustar el modelo óptimo en el conjunto de datos completo
    optimal_model = glmnet(X, y, alpha=1.0, lambda=[optimal_lambda])
    
    # Para los betas
    optimal_betas = GLMNet.coef(optimal_model)
    
    # usamos un Dict para los valores
    return Dict(
        "optimal_lambda" => optimal_lambda,
        "optimal_betas" => optimal_coefs,
        "all_lambdas" => grid,
        "all_mse" => mse_per_lambda
    )
end


lasso_cv (generic function with 1 method)

In [77]:
#9.


In [ ]:
#10

In [ ]:
#11